In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import matplotlib.animation as animation

In [ ]:
omega = 5.0

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(None, 2)),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(2)
])

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
def harmonic_oscillator(t, y):
    x, v = y
    dx_dt = v
    dv_dt = -omega**2 * x
    return [dx_dt, dv_dt]

In [ ]:
initial_conditions = [1.0, 0.0]  
sol = solve_ivp(harmonic_oscillator, [0, 10], initial_conditions, t_eval=np.linspace(0, 10, 110))

In [ ]:
X_train = []
Y_train = []

In [ ]:
for i in range(len(sol.t) - 10):
    X_train.append(sol.y[:, i:i+10].T)
    Y_train.append(sol.y[:, i+10])

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
model.fit(X_train, Y_train, epochs=100)

In [ ]:
initial_conditions_test = sol.y[:, :10].T

In [ ]:
predictions = []

In [ ]:
for _ in range(100):
    prediction = model.predict(initial_conditions_test.reshape(1, 10, 2))
    predictions.append(prediction[0])
    
    initial_conditions_test = np.roll(initial_conditions_test, -1, axis=0)
    initial_conditions_test[-1, :] = prediction

predictions = np.array(predictions)

In [ ]:
true_states = sol.y[:, 10:110].T

In [ ]:
time = np.arange(len(predictions))

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(time, predictions[:, 0], label='Predicted')
plt.plot(time, true_states[:, 0], label='True')
plt.xlabel('Time')
plt.ylabel('Position')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(time, predictions[:, 1], label='Predicted')
plt.plot(time, true_states[:, 1], label='True')
plt.xlabel('Time')
plt.ylabel('Velocity')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

In [ ]:
ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-1.5, 1.5)
ax2.set_xlim(-1.5, 1.5)
ax2.set_ylim(-1.5, 1.5)

In [ ]:
line_true, = ax1.plot([], [], '-', lw=2, alpha=0.5)
line_pred, = ax2.plot([], [], '-', lw=2, alpha=0.5)
point_true, = ax1.plot([], [], 'o', lw=2)
point_pred, = ax2.plot([], [], 'o', lw=2)

In [ ]:
ax1.set_title('True')
ax2.set_title('Predicted')

In [ ]:
x_true_data, v_true_data = [], []
x_pred_data, v_pred_data = [], []

In [ ]:
def init():
    line_true.set_data([], [])
    line_pred.set_data([], [])
    point_true.set_data([], [])
    point_pred.set_data([], [])
    return line_true, line_pred, point_true, point_pred,

In [ ]:
def animate(i):
    x_true = true_states[i, 0]
    v_true = true_states[i, 1]
    x_pred = predictions[i, 0]
    v_pred = predictions[i, 1]

    x_true_data.append(x_true)
    v_true_data.append(v_true)
    x_pred_data.append(x_pred)
    v_pred_data.append(v_pred)

    line_true.set_data(x_true_data, v_true_data)
    line_pred.set_data(x_pred_data, v_pred_data)
    point_true.set_data(x_true, v_true)
    point_pred.set_data(x_pred, v_pred)
    return line_true, line_pred, point_true, point_pred,

In [ ]:
ani = animation.FuncAnimation(fig, animate, init_func=init, frames=100, interval=100, blit=True)

In [ ]:
plt.show()